In [1]:
# Import necessary libraries
import pandas as pd
import random
import re
import os

# Load the datasets
inspired from \cite{saadany2020fake}

## Fake news
1. Satirical fake news from Al-Hudood and Al-Ahram Al-Mexici : Saadany et al. dataset
https://github.com/sadanyh/Arabic-Satirical-Fake-News-Dataset/blob/master/fake_news.zip 

In [ ]:
# Define the path where the fake news text files are stored
folder_path = './fake_news'

# Initialize an empty list to store the data
fake_news_data = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Only process .txt files
        file_path = os.path.join(folder_path, filename)
        
        # Open the text file and read its contents
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            fake_news_data.append(content)

# Create a DataFrame from the fake news text data
satirical_fake_news = pd.DataFrame(fake_news_data, columns=['text'])
satirical_fake_news['label'] = 'fake'  # Add the label


# Shuffle the dataset
satirical_fake_news = satirical_fake_news.sample(frac=1, random_state=42).reset_index(drop=True)


# Save the compiled dataset to a new CSV file
satirical_fake_news.to_csv("fake_news_dataset.csv", index=False)


num_fake_news=len(satirical_fake_news)
print(f"Sampled {num_fake_news} fake articles from Al-Hudood and Al-Ahram Al-Mexici.")
print(satirical_fake_news.head)

Sampled 3185 fake articles from Al-Hudood and Al-Ahram Al-Mexici.
<bound method NDFrame.head of                                                    text label
0     الأسد يختبئ داخل بيوت النازحين ليصطادهم عند عو...  fake
1     مشج ع رياضي ينشغل بمتابعة المباراة عن ترديد ال...  fake
2     كاريكاتير فارس قره بيت   السلطة تدعو لفك الارت...  fake
3     الحكومة الأمريكية تحيل جواسيسها في المنطقة الع...  fake
4     ترشيح لجائزة أكثر خبر مضل   كلب  يتحول  إلى دب...  fake
...                                                 ...   ...
3180  ترشيح لجائزة الخبر الأكثر إنحيازا    إسرائيل ت...  fake
3181  ترشيح لجائزة الخاصة   فتيات يأكلن ويشربن في ال...  fake
3182  العبادي يحذ ر المواطنين من الاستمرار بالتظاهر ...  fake
3183  سائق باص يغادر المحطة رغم وجود فسحة تسع لراكب ...  fake
3184  قال الصحفي الشهير مصطفى بكرى أنه لولا وجود الس...  fake

[3185 rows x 2 columns]>


## Real news
2. True news from BBC and CNN

- method in https://arxiv.org/pdf/2011.00452 

"The real news dataset comprises 3185 articles from the BBC-News Arabic and CNN-Arabic news datasets which are available on the Sourceforge part of the Arabic Computational Linguistics project (https://sourceforge.net/projects/ar-text-mining/files/Arabic-Corpora/). 

To ensure that the fake and real datasets talk about similar themes, we chose the articles that belong to the politics section in the two real news corpora."


- Data processing done manually:

Only took "middle east" and "world" news folders from BBC and CNN news folders and put them together as the real_news dataset

In [ ]:
# Define the path where the fake news text files are stored
folder_path = './real_news'

# Initialize an empty list to store the data
real_news_data = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Only process .txt files
        file_path = os.path.join(folder_path, filename)
        
        # Open the text file and read its contents
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            real_news_data.append(content)


# Create a DataFrame from the fake news text data
true_news_sample = pd.DataFrame(real_news_data, columns=['text'])
true_news_sample['label'] = 'real'  # Add the label


# Shuffle the dataset
true_news_sample = true_news_sample.sample(frac=1, random_state=42).reset_index(drop=True)

# Reduce the number of real news articles to match the number of fake news articles
if len(true_news_sample) > num_fake_news:
    true_news_sample = true_news_sample.head(num_fake_news)


# Save the compiled dataset to a new CSV file
true_news_sample.to_csv("real_news_dataset.csv", index=False)


# Display the shape and first few rows of the compiled dataset
num_true_news=len(true_news_sample)
print(f"Sampled {num_true_news} real articles from BBC and CNN from SourceForge.")
print(true_news_sample.head)

Sampled 3185 real articles from BBC and CNN from SourceForge.
<bound method NDFrame.head of                                                    text label
0                                   ‭BBC Arabic‬ - ‮...  real
1                                   ‭BBC Arabic‬ - ‮...  real
2               CNNArabic.com - صحف: "طوابير خامسة" ...  real
3                                ‭BBC Arabic‬ - ‮الش...  real
4                                ‭BBC Arabic‬ - ‮الش...  real
...                                                 ...   ...
3180            CNNArabic.com - أمير قطر يجدد الدعوة...  real
3181            CNNArabic.com - مدير CIA السابق ينتق...  real
3182            CNNArabic.com - "حماس": شقيق سامي أب...  real
3183            CNNArabic.com - موسيقيون يوقعون عريض...  real
3184                                ‭BBC Arabic‬ - ‮...  real

[3185 rows x 2 columns]>


### Cleaning of real data
real dataset seems messy but fake dataset looks good

In [4]:
# Cleaning the real news dataset
# Step 1: Load the Data
true_news_sample_cleaned = pd.read_csv('real_news_dataset.csv')

# Step 2: Clean the Text
def clean_text(text):

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove leading/trailing whitespace
    text = text.strip()

    # Keep only Arabic letters, spaces, and some punctuation
    text = re.sub(r'[^أ-ي\s،؛؟.]+', '', text) 

    # Normalize spaces (replace multiple spaces with a single space)
    text = re.sub(r'\s+', ' ', text)
    
    return text

# Apply the cleaning function to the 'text' column
true_news_sample_cleaned['text'] = true_news_sample_cleaned['text'].apply(clean_text)

# Step 3: Save the Cleaned Data
true_news_sample_cleaned.to_csv('cleaned_real_news.csv', index=False)

# Display the shape and first few rows of the compiled dataset
num_real_news_df = len(true_news_sample_cleaned)
print(f"Sampled {num_real_news_df} real articles (CLEANED) from BBC and CNN from SourceForge.")
print(true_news_sample_cleaned.head)

Sampled 3185 real articles (CLEANED) from BBC and CNN from SourceForge.
<bound method NDFrame.head of                                                    text label
0      الشرق الأوسط الأراضي الفلسطينية في العقد الأو...  real
1      الشرق الأوسط البرلمان العراقي يطالب باستجواب ...  real
2     . صحف طوابير خامسة بالكويت.. ونكتة تغضب السودا...  real
3      الشرق الأوسط الاسد يدعو فرنسا للعب دور أكبر ف...  real
4      الشرق الأوسط بالصور الطريق الى مواقع التمرد ب...  real
...                                                 ...   ...
3180  . أمير قطر يجدد الدعوة لعقد قمة عربية طارئة أم...  real
3181  . مدير السابق ينتقد كشف أوباما لمذكرات عن تعذي...  real
3182  . حماس شقيق سامي أبو زهري عذب حتى الموت بمصر ا...  real
3183  . موسيقيون يوقعون عريضة لكشف استخدام أعمالهم ب...  real
3184   العالم زعيم المعارضة في اليابان فوزنا يمثل ثو...  real

[3185 rows x 2 columns]>


## Tweets (real and fake)
3. Tweets from (Alyoubi et al., 2023): https://www.mdpi.com/2076-3417/13/14/8209

In [5]:
# tweets = pd.read_csv('path_to_tweets.csv')  # Load the tweets dataset
# tweets['label'] = tweets['label'].map({0: 'fake', 1: 'real'})  # Assuming 0 is fake and 1 is real in the tweet dataset

## Combine all datasets into one

In [ ]:
# Combine all datasets into one
# final_dataset = pd.concat([satirical_fake_news, true_news_sample, tweets], ignore_index=True)
final_dataset = pd.concat([satirical_fake_news, true_news_sample_cleaned], ignore_index=True)

# Shuffle the final dataset
final_dataset = final_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the shape and first few rows of the compiled dataset
print(f"Final dataset shape: {final_dataset.shape}")
print(final_dataset.head)

# Save the compiled dataset to a new CSV file
final_dataset.to_csv('compiled_real_fake_news_dataset.csv', index=False)

Final dataset shape: (6370, 2)
<bound method NDFrame.head of                                                    text label
0      الشرق الأوسط الترقب يسود في الخليل بعد مواجها...  real
1     فتاة تقبل بشقة واحدة وخاتم بألماسة وحيدة تقدير...  fake
2        محافظ القليوبية  آلاف جنيه لكل مدينة وقرية ...  fake
3     السيسي يؤك د أن ه يعتقل منافسيه على الرئاسة لي...  fake
4      الشرق الأوسط العراق مقتل واصابة شخصا في حوادث...  real
...                                                 ...   ...
6365   الشرق الأوسط العفو الدولية تحذر من نقل معسكر ...  real
6366   الشرق الأوسط الحكم بالإعدام على هنديا في الإم...  real
6367  . محكمة سعودية تصدر أحكاما بحق عضوا في تنظيم ا...  real
6368   الشرق الأوسط وفد إسرائيلي ينقب عن رفات جنود إ...  real
6369  دليل   كيف تستغني عن المنطق في حياتك اليومية  ...  fake

[6370 rows x 2 columns]>
